In [334]:
import pandas as pd
import numpy as np

<img src="cat.png"  width="500"/>

We will use one-hot encoding to encode the categorical features. They will be as follows:

- Ear Shape: Pointy = 1, Floppy = 0
- Face Shape: Round = 1, Not Round = 0
- Whiskers: Present = 1, Absent = 0

In [249]:
X_train = np.array(
    [[1, 1, 1],
    [0, 0, 1],
    [0, 1, 0],
    [1, 0, 1],
    [1, 1, 1],
    [1, 1, 0],
    [0, 0, 0],
    [1, 1, 0],
    [0, 1, 0],
    [0, 1, 0]]
)

y_train = np.array([1, 1, 0, 0, 1, 1, 0, 1, 0, 0])

In [317]:
# Let,s Get the shape of the x_train, y_train
print(f"X_train Shape: {X_train.shape}")
print(f"X_train Shape: {y_train.shape}")

X_train Shape: (10, 3)
X_train Shape: (10,)


In [251]:
# Let,s print the first example to see
print(X_train[0])

[1 1 1]


This means that the first example has a pointy ear shape, round face shape and it has whiskers.

On each node, we compute the information gain for each feature, then split the node on the feature with the higher information gain, by comparing the entropy of the node with the weighted entropy in the two splitted nodes. 

So, the root node has every animal in our dataset. Remember that $p_1^{node}$ is the proportion of positive class (cats) in the root node. So

$$p_1^{node} = \frac{5}{10} = 0.5$$

Now let's write a function to compute the entropy.

In [311]:
def entropy(y):
    if len(y) == 0:
        return 0
    p1 = len(y[y == 1]) / len(y) 
    if p1 == 0 or p1 == 1:
        return 0
    return -p1 * np.log2(p1) - (1 - p1) * np.log2(1 - p1)
# Let us try with y and it should print 1
print(entropy(y_train))

1.0


In [257]:
def split_indices(X, node_indices, index_feature):
    left_indices = []
    right_indices = []

    for i in node_indices:
        if X[i][index_feature] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)
    return left_indices, right_indices

So, if we choose Ear Shape to split, then we must have in the left node (check the table above) the indices:

$$0 \quad 3 \quad 4 \quad 5 \quad 7$$

and the right indices, the remaining ones.

In [261]:
node_indices = range(10)
split_indices(X_train, node_indices, 0)

([0, 3, 4, 5, 7], [1, 2, 6, 8, 9])

In [227]:
left_indices, right_indices = split_indices(X_train, 0)
weighted_entropy(X_train, y_train, left_indices, right_indices)

0.7219280948873623

In [263]:
def information_gain(X, y, node_indices, feature):

    left_indices, right_indices = split_indices(X, node_indices, feature)
    
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]

    
    node_entropy = entropy(y_node)
    left_entropy = entropy(y_left)
    right_entropy = entropy(y_right)

    w_left = len(X_left) / len(X_node)
    w_right = len(X_right) / len(X_node)
    
    weighted_entropy = (w_left * left_entropy) + (w_right * right_entropy)
    info_gain = node_entropy - weighted_entropy
    return info_gain

In [231]:
information_gain(X_train, y_train, left_indices, right_indices)

0.2780719051126377

Now, let's compute the information gain if we split the root node for each feature:

In [265]:
for i, feature_name in enumerate(['Ear Shape', 'Face Shape', 'Whiskers']):
    left_indices, right_indices = split_indices(X_train, node_indices, i)
    info_gain = information_gain(X_train, y_train, node_indices, i)
    print(f"Feature: {feature_name}, information gain if we split the root node using this feature: {info_gain:.2f}")

Feature: Ear Shape, information gain if we split the root node using this feature: 0.28
Feature: Face Shape, information gain if we split the root node using this feature: 0.03
Feature: Whiskers, information gain if we split the root node using this feature: 0.12



This function selects the best feature to split on by computing the **information gain** for each feature in the dataset, and get the highest information gain.  


In [291]:
def get_best_gain(X, y, node_indices):

    best_feature = -1
    max_info_gain = 0

    for i in range(X.shape[1]):
        left_indices, right_indices =  split_indices(X, node_indices, i)
        info_gain = information_gain(X, y, node_indices, i)
        if info_gain > max_info_gain:
            best_feature = i
            max_info_gain = info_gain

    return best_feature, max_info_gain

In [322]:
features = ['Ear Shape', 'Face Shape', 'Whiskers']
best_feature, best_gain = get_best_gain(X_train, y_train, node_indices)
print(f"Best feature to split on: {features[best_feature]}, with information gain: {best_gain}")

Best feature to split on: Ear Shape, with information gain: 0.2780719051126377



This recursive function builds a decision tree by splitting the data at each node based on the best feature. It continues splitting the data recursively until the maximum depth is reached.
The function splits the data at each node based on the best feature, prints the details about the splits, and recursively builds the tree for the left and right child nodes.

In [330]:
def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
   
    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature, _ = get_best_gain(X, y, node_indices) 
    
    formatting = "-"*current_depth
    print(f"{formatting} Depth {current_depth}, {branch_name}: Split on feature: {best_feature}")
    # Split the dataset at the best feature
    left_indices, right_indices = split_indices(X, node_indices, best_feature)
    
    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)

In [332]:
build_tree_recursive(X_train, y_train, node_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 0
- Depth 1, Left: Split on feature: 1
  -- Left leaf node with indices [0, 4, 5, 7]
  -- Right leaf node with indices [3]
- Depth 1, Right: Split on feature: 2
  -- Left leaf node with indices [1]
  -- Right leaf node with indices [2, 6, 8, 9]
